In [6]:
from ultralytics import YOLO
import pandas as pd
import cv2
import os
import json

In [7]:
yolos = [
    "model_0",
    "model_1",
    "model_2",
]

In [8]:
def yolo_test(key, model, fold):
    #test = pd.read_csv(f"results/fold_{fold}_val", index_col=0)
    #test.rename(columns={"img": "ID"}, inplace=True)

    test = pd.read_csv(f"CAA01/Test.csv")
    submission = pd.read_csv("CAA01/Zindi/SampleSubmission.csv")

    #test = pd.read_csv(f"CAA01/Train.csv")
    #test = test.groupby("ID").sum().reset_index()

    def resize_image(name, size=(512, 512)):
        image_file = f'CAA01_images/{name.replace(".png", "")}.jpg'
        if os.path.exists(f"yoloM/{name}.png"):
            return
        
        image = cv2.imread(image_file)
        resized_image = cv2.resize(image, size)
        cv2.imwrite(f"yoloM/{name}.png", resized_image)

    for i, row in test.iterrows():

        resize_image(row["ID"])
        results = model(f"yoloM/{row['ID']}.png")
        result = results[0]

        # Count occurrences of each detected class
        class_counts = {}

        for cls in result.boxes.cls.tolist():  # Get list of detected class indices
            class_counts[int(cls)] = class_counts.get(int(cls), 0) + 1

        # Map class indices to class names
        class_names = model.names  # Dictionary mapping class index to name
        named_counts = {class_names[k]: v for k, v in class_counts.items()}

        if "thermal" in named_counts:
            submission.loc[submission["ID"] == f'{row["ID"]}_boil', "Target"] = named_counts["thermal"]
            #test.loc[test["ID"] == f'{row["ID"]}', "boil"] = named_counts["thermal"]
        
        if "photovoltaic" in named_counts:
            submission.loc[submission["ID"] == f'{row["ID"]}_pan', "Target"] = named_counts["photovoltaic"]
            #test.loc[test["ID"] == f'{row["ID"]}', "pan"] = named_counts["photovoltaic"]
    
    submission.to_csv(f"{key}_test.csv", index=False)
    #test.to_csv(f"{key}_train.csv", index=False)

In [9]:

for yolo in yolos:
    #model = YOLO(f"/Users/hugover/Desktop/CAA01/ModelBuild/YOLOv8L 06 (id)/results/{yolo}/weights/best.pt")
    fold = yolo.split("_")[1]

    #yolo_test(yolo, model, fold)

In [10]:
df01 = pd.read_csv("model_0_train.csv")[["ID", "boil", "pan"]]
df02 = pd.read_csv("model_1_train.csv")[["ID", "boil", "pan"]]
df03 = pd.read_csv("model_2_train.csv")[["ID", "boil", "pan", "boil_nbr", "pan_nbr"]]

df01.rename(columns={"boil": "boil0", "pan": "pan0"}, inplace=True)
df02.rename(columns={"boil": "boil1", "pan": "pan1"}, inplace=True)
df03.rename(columns={"boil": "boil2", "pan": "pan2"}, inplace=True)

merge = pd.merge(df01, df02, on="ID", how="outer")
merge = pd.merge(merge, df03, on="ID", how="outer")
merge.fillna(0, inplace=True)

merge["boil"] = round(merge["boil0"] + merge["boil1"] + merge["boil2"]) / 3
merge["pan"] = round(merge["pan0"] + merge["pan1"] + merge["pan2"]) / 3

merge["mae_boil"] = abs(merge["boil"] - merge["boil_nbr"])
merge["mae_pan"] = abs(merge["pan"] - merge["pan_nbr"])

mae = (merge["mae_boil"].mean() + merge["mae_pan"].mean()) / 2
mae

1.4329206924315618

In [11]:
dftest01 = pd.read_csv("model_0_test.csv")
dftest02 = pd.read_csv("model_1_test.csv")
dftest03 = pd.read_csv("model_2_test.csv")
dftest01.rename(columns={"Target": "0"}, inplace=True)
dftest02.rename(columns={"Target": "1"}, inplace=True)
dftest03.rename(columns={"Target": "2"}, inplace=True)

merge = pd.merge(dftest01, dftest02, on="ID", how="outer")
merge = pd.merge(merge, dftest03, on="ID", how="outer")
merge.fillna(0, inplace=True)

merge["Target"] = round(merge["0"] + merge["1"] + merge["2"]) / 3

merge[["ID", "Target"]].to_csv("submission.csv", index=False)